Hello Ben,

I enjoyed solving the problem and I want to utilise the 24 hours you've given me. Kindly view this notebook as a supplement to the main submission. I start with the cleaned version of the data.

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

D = pd.read_csv('clean_diabetes.csv')

In [8]:
D.describe()

,Unnamed: 0,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,383.500000,3.785156,120.946615,69.140625,20.944010,77.072917,31.980469,0.467483,33.769531,0.348958
std,221.846794,3.291875,30.990784,19.068969,15.527097,110.316623,7.735926,0.323716,15.626516,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,-1.000000,0.000000
25%,191.750000,1.000000,100.000000,64.000000,0.000000,0.000000,27.500000,0.251000,24.000000,0.000000
50%,383.500000,3.000000,117.000000,72.000000,23.000000,36.000000,32.000000,0.372500,29.000000,0.000000
75%,575.250000,6.000000,139.000000,80.000000,32.000000,122.000000,36.225000,0.602000,40.000000,1.000000
max,767.000000,17.000000,198.000000,122.000000,99.000000,846.000000,67.100000,2.420000,200.000000,1.000000


In [9]:
D = D.drop(columns=['Unnamed: 0'])
D = D.loc[D['Insulin'] < 400] # Remove outliers.
D.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,750.00000,750.000000,750.000000,750.000000,750.000000,750.000000,750.000000,750.000000,750.000000
mean,3.79600,120.124000,69.016000,20.616000,66.373333,31.891600,0.466344,33.782667,0.342667
std,3.29457,30.675429,19.213987,15.493976,85.567449,7.740028,0.320410,15.686157,0.474918
min,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,-1.000000,0.000000
25%,1.00000,100.000000,64.000000,0.000000,0.000000,27.500000,0.251000,24.000000,0.000000
50%,3.00000,117.000000,72.000000,23.000000,36.000000,32.000000,0.372500,29.000000,0.000000
75%,6.00000,138.000000,78.000000,32.000000,115.000000,36.100000,0.600000,40.000000,1.000000
max,17.00000,198.000000,122.000000,99.000000,392.000000,67.100000,2.420000,200.000000,1.000000


We redefine helped functions. It would have been nice if I could have imported them as a module!!

In [25]:
def trn_tst_split(data, trn_pct):
    """Split the data set into training and test.
    
        data: the data frame to be split.
        trn_pct: % of training data. Must be between 0 and 1.
        
        Returns a list of two data frames - training and test.
    """
    random.seed(12111842)
    N = data.shape[0]
    n_train = int(N * trn_pct)
    trn_indices = set(random.sample(range(N), n_train))
    tst_indices = set(range(N)) - trn_indices
    
    assert len(trn_indices.intersection(tst_indices)) == 0
    assert len(trn_indices) + len(tst_indices) == N
    
    # Convert them to lists.
    trn_indices = [i for i in trn_indices]
    tst_indices = [i for i in tst_indices]
    
    trn_data = data.iloc[trn_indices, :]
    tst_data = data.iloc[tst_indices, :]
    
    assert trn_data.shape[0] + tst_data.shape[0] == data.shape[0]
    
    return [trn_data, tst_data]


def print_cm_results(cm):
    """Print diagnostics from the confusion matrix."""
    recall = cm[0, 0]/(cm[0, 0] + cm[1, 0])
    precision = cm[0, 0]/(cm[0, 0] + cm[0, 1])
    specificity = cm[1, 1]/(cm[1, 0] + cm[1, 1])
    f1_score = 2*recall*precision/(recall + precision)
    accuracy = np.trace(cm)/np.sum(cm)
    
    print(f'% +ves correctly predicted (recall) = {round(recall * 100, 2)}')
    print(f'% +ves detected out of all (precision) = {round(precision * 100, 2)}')
    print(f'% -ves detected out of all (specificity) = {round(specificity * 100, 2)}')
    print(f'f1 score = {round(f1_score, 2)}')
    print(f'accuracy = {round(accuracy * 100, 2)}')
    
def get_model_matrices(D, formula):
    return dmatrices(formula, data=D, return_type='dataframe')

def prob_to_outcome(y, threshold = 0.5):
    """Converts probability to a class depending on the threshold."""
    if y < threshold:
        return 0
    else:
        return 1
    
def build_cm(yt, yp):
    """Generates confusion matrix from actual and predicted values."""
    yt['predicted'] = [prob_to_outcome(y) for y in yp]
    yt.columns = ['actual', 'predicted']
    yt['actual'] = yt['actual'].apply(lambda f: int(f))
    cm = pd.crosstab(yt['actual'], yt['predicted']).to_numpy()
    
    return cm    

Perhaps there is a library in Python to carry out cross-validation. But I don't yet know about it. Therefore, I implemented a few functions to carry out 30-fold cross validation. We have 750 observations. I build a model using 725 of them and use 25 to test. I choose the 25 observations to test sequentially. In the first iteration rows 0-24 are test rows, in the second one 25-49 are test rows, and so on. In each iteration, I compute the accuracy of the model and the model coefficients. I save them in two lists.

Finally, I will take an average of the model coefficients.


In [34]:
boundaries = np.linspace(0, D.shape[0], 26)
all_params = []
all_accuracies = []

for i in range(len(boundaries) - 1):
    l = int(boundaries[i])
    r = int(boundaries[i+1])
    print(f'{l}, {r}')
    D_test = D.iloc[l:r, :]
    D_train = pd.concat([D.iloc[0:l, :], D.iloc[r:D.shape[0], :]], axis=0)
    yt, Xt = get_model_matrices(D_test, f_v2)
    yn, Xn = get_model_matrices(D_train, f_v2)
    model = sm.Logit(yn, Xn)
    results = model.fit()
    yp = results.predict(Xt)
    cm = build_cm(yt, yp)
    
    all_accuracies.append(np.trace(cm)/np.sum(cm))
    all_params.append(results.params)
    

0, 30
Optimization terminated successfully.
         Current function value: 0.470162
         Iterations 6
30, 60
Optimization terminated successfully.
         Current function value: 0.479003
         Iterations 6
60, 90
Optimization terminated successfully.
         Current function value: 0.484953
         Iterations 6
90, 120
Optimization terminated successfully.
         Current function value: 0.483766
         Iterations 6
120, 150
Optimization terminated successfully.
         Current function value: 0.479860
         Iterations 6
150, 180
Optimization terminated successfully.
         Current function value: 0.485337
         Iterations 6
180, 210
Optimization terminated successfully.
         Current function value: 0.478708
         Iterations 6
210, 240
Optimization terminated successfully.
         Current function value: 0.476786
         Iterations 6
240, 270
Optimization terminated successfully.
         Current function value: 0.477413
         Iterations 6
270, 300


In [36]:
print([round(a, 2) for a in all_accuracies])

[0.6, 0.7, 0.87, 0.83, 0.77, 0.9, 0.7, 0.67, 0.7, 0.67, 0.7, 0.8, 0.77, 0.8, 0.77, 0.7, 0.87, 0.77, 0.8, 0.93, 0.77, 0.63, 0.77, 0.77, 0.77]


In [39]:
all_params_df = pd.concat(all_params, axis=1).reset_index()

In [40]:
all_params_df

,index,0,1,2,3,4,5,6,7,8,...,15,16,17,18,19,20,21,22,23,24
0,Intercept,-8.904226,-8.743452,-8.459655,-8.458774,-8.554322,-8.486079,-8.615895,-8.696815,-8.685007,...,-8.813801,-8.571501,-8.508374,-8.425819,-8.277207,-8.644413,-8.794412,-8.755106,-8.539364,-8.544492
1,Pregnancies,0.132456,0.131429,0.132501,0.131689,0.131788,0.126393,0.127783,0.126877,0.128656,...,0.135763,0.127942,0.119695,0.142992,0.127544,0.122811,0.139749,0.125484,0.125112,0.143938
2,Glucose,0.035006,0.036191,0.034707,0.034253,0.035055,0.034378,0.034825,0.035551,0.035793,...,0.036372,0.036113,0.035323,0.033959,0.033001,0.035289,0.034824,0.033840,0.035002,0.033835
3,BMI,0.087112,0.079521,0.077771,0.078560,0.079265,0.080717,0.082737,0.080584,0.080995,...,0.081600,0.076312,0.078556,0.078875,0.078307,0.081696,0.086256,0.088747,0.079983,0.081103
4,DiabetesPedigreeFunction,1.034731,1.064587,0.933277,0.981074,0.928960,0.911971,0.871549,1.012282,0.924514,...,0.992899,0.955082,0.955910,0.939139,0.982489,0.981492,0.954487,1.050669,0.926435,0.966304


In [62]:
from scipy.stats import sem, t

def get_mean_with_ci(X):
    m = np.mean(X)
    conf = 0.95
    N = len(X)
    h = sem(X) * t.ppf((1 + conf)/2, N - 1)
    
    return (m, m - h, m + h)

The final results of the exercise are the outputs of the two cells below. In particular, when we put the model in production, we will use the mean values of the coefficients printed in the last cell. The 95% confidence interval of the model's accuracy is printed in the cell below.

In [77]:
(m, l, r) = get_mean_with_ci(all_accuracies)
print(f'Accuracies mean = {round(m, 2)}, C.I. = [{round(l, 2)}, {round(r, 2)}]')

Accuracies mean = 0.76, C.I. = [0.73, 0.79]


In [74]:
all_intervals = []
all_rows = []

for i in range(all_params_df.shape[0]):
    row = all_params_df.iloc[i, :].to_numpy()
    name = row[0]
    all_rows.append(row)
    info = get_mean_with_ci(row[1:len(row)])
    print(f'{name}: coeff-mean = {round(info[0], 3)}, C.I. = [{round(info[1], 3)}, {round(info[2], 3)}]')    

Intercept: coeff-mean = -8.59, C.I. = [-8.651, -8.529]
Pregnancies: coeff-mean = 0.132, C.I. = [0.129, 0.134]
Glucose: coeff-mean = 0.035, C.I. = [0.034, 0.035]
BMI: coeff-mean = 0.081, C.I. = [0.079, 0.082]
DiabetesPedigreeFunction: coeff-mean = 0.972, C.I. = [0.946, 0.997]
